In [1]:
#Imports
import sys
sys.path.append('../../python/')
import NGC5533_functions as nf

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import lmfit as lm
import dataPython as dp

from datetime import datetime
import time as t

In [2]:
starttime = t.time()

data = dp.getXYdata_wXYerr('../data/100kpc_data.txt')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])

#change r_dat so it's strictly increasing
r_dat, v_dat, v_err0, v_err1 = (list(a) for a in zip(*sorted(zip(r_dat, v_dat, v_err0, v_err1))))
#converting v_err1 to an array
v_err1 = np.asarray(v_err1)
#Express as weights
weighdata = 1/v_err1

In [3]:
#keeping others but holding them constant
def f(r,M,n,re):
     return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 
                    + nf.h_v(r,load=True,path='../')**2 
                    + nf.b_v(r,n,re,load=True,path='../')**2 
                    + nf.d_v(r,load=True,path='../')**2)
#Only Black Hole and Bulge
def bh_b(r,M,n,re):
    return np.sqrt(nf.bh_v(r,M,load=True,path='../')**2 + nf.b_v(r,n,re,load=True,path='../')**2)

In [ ]:
#Fit, Keeping rest constant

#Setup
l_mod = lm.Model(f)
l_params = l_mod.make_params()
#Black Hole
l_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Bulge
l_params.add('n', value=nf.n_c, min=0) 
l_params.add('re', value=nf.re_c, min=0)
#Do fit
l_fit = l_mod.fit(v_dat,l_params,r=r_dat,weights=weighdata)

../../python/NGC5533_functions.py:182: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return si.quad(h, 0, r, args=(r,n,re))[0]


In [ ]:
#Fit, BH and B Only

#Setup
bhb_mod = lm.Model(bh_b)
bhb_params = l_mod.make_params()
#Black Hole
bhb_params.add('M', value=nf.Mbh_def, min=0) #Mass
#Bulge
bhb_params.add('n', value=nf.n_c, min=0) 
bhb_params.add('re', value=nf.re_c, min=0)
#Do fit
bhb_fit = bhb_mod.fit(v_dat,bhb_params,r=r_dat,weights=weighdata)

In [ ]:
#Define curves to plot

l_dict = l_fit.best_values
l_M = l_dict['M']
l_n = l_dict['n']
l_re = l_dict['re']
l_curve = f(r_dat,l_M,l_n,l_re)

bhb_dict = bhb_fit.best_values
bhb_M = bhb_dict['M']
bhb_n = bhb_dict['n']
bhb_re = bhb_dict['re']

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,125)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_b(r_dat,l_M,l_n,l_re),'b--',label='Black Hole + Disk, from All Components')
plt.plot(r_dat,bh_b(r_dat,nf.Mbh_def,nf.n_c,nf.re_c),'g-.',label='Black Hole + Disk (Defaults)')
plt.plot(r_dat,bh_b(r_dat,d_h,d_d_rho00),label='Fit Black Hole + Disk Only')

plt.legend()
plt.show()

print('Fit information for all-component fit:')
l_fit

In [ ]:
fig = plt.figure(figsize=(9.0,8.0))

plt.xlim(0,25)

plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
plt.plot(r_dat,l_curve,'b-',label='Fit with All Components')
plt.plot(r_dat,bh_b(r_dat,l_M,l_n,l_re),'b--',label='Black Hole + Disk, from All Components')
plt.plot(r_dat,bh_b(r_dat,nf.Mbh_def,nf.n_c,nf.re_c),'g-.',label='Black Hole + Disk (Defaults)')
plt.plot(r_dat,bh_b(r_dat,d_h,d_d_rho00),label='Fit Black Hole + Disk Only')


plt.legend()
plt.show()

print('Fit information for two-component fit')
d_fit

In [ ]:
endtime = t.time()
print('Total Time:')
print(str(endtime - starttime)+' seconds')
print('')

print('Timestamp:')
print(datetime.now())